In [1]:
# Need this version because there's an issue with audio for the latest version
!pip install git+https://github.com/huggingface/transformers
!pip install torch torchvision torchaudio
!pip install accelerate -U
!pip install datasets
!pip install gradio
!pip install pandas
!pip install numpy

  Cloning https://github.com/huggingface/transformers to /private/var/folders/wc/wntr3k4n1_xdk3mtbmwhbzjc0000gn/T/pip-req-build-ie_f5tkw
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /private/var/folders/wc/wntr3k4n1_xdk3mtbmwhbzjc0000gn/T/pip-req-build-ie_f5tkw
  Resolved https://github.com/huggingface/transformers to commit d9850abd407821fb9f4ad5aabb206adad3610f75
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 19.0 MB/s eta 0:00:00a 0:00:01
  Created wheel for transformers: filename=transformers-4.41.0.dev0-py3-none-any.whl size=8998772 sha256=3d756222c998635e53714a8d5953a80fd6de7ef5881ddd8826a28de647e23cc8
  Stored in directory: /private/var/folders/wc/wntr3k4n1_xdk3mtbmwhbzjc0000gn/T/pip-ephem-wheel-cache-390hp_kj/wheels/04/a3/f1/b88775f8e1665827525b19ac7590250f1038d947067beba9fb
Successfu

In [2]:
#Loading data from GTZAN (should have 999 tracks since one is corrupted)
# from datasets import load_dataset

# gtzan = load_dataset("marsyas/gtzan", "all")
# gtzan
%matplotlib inline
import numpy as np
import pandas as pd

df = pd.read_json("../updated_tracks_full.json")
mixes_df = pd.json_normalize(df["mixes"])
mixes_df.head(5)

# Create a multiple dimensional array for the number of mixes we have available to us
mixes = []
for i, row in mixes_df.iterrows():
    mixes.append(row["tracks"])
    
track_list = []
for mix in mixes: 
    for track in mix:
        if "track-audio-features" in track:
            track_list.append(track)
       
track_list_df = pd.DataFrame(track_list)
track_list_df.head(5)
# pd.DataFrame(tracks)

,track-name,track-type,track-instrument-subtype,track-instrument-type,channel-mode,parameters,track-audio-features
0,Kick In,AUDIO,KICK,DRUMS,MONO,"{'gain': -5.9, 'pan': [0], 'gate': [{'name': '...",{'track-audio-path': './audio/Vermont/01 K in....
1,Kick Out,AUDIO,KICK,DRUMS,MONO,"{'gain': -5.9, 'pan': [0], 'gate': [{'name': '...",{'track-audio-path': './audio/Vermont/02 K out...
2,Snare Top,AUDIO,SNARE,DRUMS,MONO,"{'gain': -9.9, 'pan': [0], 'eq': [{'type': 'HP...",{'track-audio-path': './audio/Vermont/03 S top...
3,Snare Bott,AUDIO,SNARE,DRUMS,MONO,"{'gain': -9.5, 'pan': [0], 'eq': [{'type': 'HP...",{'track-audio-path': './audio/Vermont/04 S bot...
4,Rack_15,AUDIO,TOM,DRUMS,MONO,"{'gain': -2.3, 'pan': [26], 'eq': [{'type': 'H...",{'track-audio-path': './audio/Vermont/05 R tom...


In [24]:
# Define your list of effects (order matters!)
effects = ['eq', 'compression', 'reverb', 'delay', 'gate', 'limiter', 'phaser', 'flanger', 'chorus']

def process_parameters_cell(cell):
    effect_code = 0  # Start with no effects
    parameters_dict = cell

    for i, effect in enumerate(effects):
        if parameters_dict.get(effect):
            effect_code |= (1 << i)  # Set the corresponding bit

    return effect_code

track_list_df['effects'] = track_list_df['parameters'].apply(process_parameters_cell)

In [26]:
def decode_effect_code(code):
    active_effects = []
    for i, effect in enumerate(effects):
        if code & (1 << i):  # Check if the bit is set
            active_effects.append(effect)
    return active_effects

In [30]:
print(decode_effect_code(track_list_df.iloc[5]['effects']))

['eq', 'reverb']


In [25]:
track_list_df.head(10)

,track-name,track-type,track-instrument-subtype,track-instrument-type,channel-mode,parameters,track-audio-features,effects
0,Kick In,AUDIO,KICK,DRUMS,MONO,"{'gain': -5.9, 'pan': [0], 'gate': [{'name': '...",{'track-audio-path': './audio/Vermont/01 K in....,256
1,Kick Out,AUDIO,KICK,DRUMS,MONO,"{'gain': -5.9, 'pan': [0], 'gate': [{'name': '...",{'track-audio-path': './audio/Vermont/02 K out...,256
2,Snare Top,AUDIO,SNARE,DRUMS,MONO,"{'gain': -9.9, 'pan': [0], 'eq': [{'type': 'HP...",{'track-audio-path': './audio/Vermont/03 S top...,5
3,Snare Bott,AUDIO,SNARE,DRUMS,MONO,"{'gain': -9.5, 'pan': [0], 'eq': [{'type': 'HP...",{'track-audio-path': './audio/Vermont/04 S bot...,5
4,Rack_15,AUDIO,TOM,DRUMS,MONO,"{'gain': -2.3, 'pan': [26], 'eq': [{'type': 'H...",{'track-audio-path': './audio/Vermont/05 R tom...,5
5,Floor,AUDIO,TOM,DRUMS,MONO,"{'gain': -2.5, 'pan': [-26], 'eq': [{'type': '...",{'track-audio-path': './audio/Vermont/06 Fl to...,5
6,OH Ride_15,AUDIO,RIDE,DRUMS,MONO,"{'gain': -17.5, 'pan': [0], 'eq': [{'type': 'H...",{'track-audio-path': './audio/Vermont/08 OH ri...,1
7,OH Coles_15,AUDIO,HI-HAT,DRUMS,MONO,"{'gain': -17.5, 'pan': [-64], 'eq': [{'type': ...",{'track-audio-path': './audio/Vermont/09 OH c....,1
8,OH_15 S,AUDIO,OVERHEAD,DRUMS,MONO,"{'gain': -17.5, 'pan': [63], 'eq': [{'type': '...",{'track-audio-path': './audio/Vermont/07 OH hh...,1
9,Room,AUDIO,OVERHEAD,DRUMS,MONO,"{'gain': -17.1, 'pan': [-64], 'eq': [{'type': ...",{'track-audio-path': './audio/Vermont/10 Rm hh...,1


In [ ]:
# # Make sure the sample rate is what we expect
# import wave
# import soundfile as sf

# for index, row in track_list_df.iterrows():
#     audio_path = '.' + str(row['track-audio-features']['track-audio-path'])
#     data, samplerate = sf.read(audio_path, dtype='float32')
#     print(f"Sample rate: {samplerate}")
#     print(f"Shape: {data}")

In [ ]:
# Need to split for testing and training
tracks_train=track_list_df.sample(frac=0.8,random_state=200)
tracks_test=track_list_df.drop(tracks_train.index)

tracks_test

In [ ]:
tracks_train['track-instrument-subtype']

In [ ]:
### Already a string so I don't think I need to do this??
# id2label_fn = gtzan["train"].features["genre"].int2str
# id2label_fn(gtzan["train"][0]["genre"])

# id2label_fn = tracks_train['track-instrument-subtype']
# id2label_fn(tracks_train['track-instrument-subtype'][0])

In [ ]:
import gradio as gr
import os

## This is the correct one to use now
# def generate_audio():
#     random_index = np.random.randint(len(tracks_test))
#     example = tracks_test.iloc[random_index]
#     original_path = '.' + example["track-audio-features"]["track-audio-path"]
#     directory, filename = os.path.split(original_path)
#     dir_components = directory.split(os.sep)
#     dir_components.insert(2, "Truncated")
#     new_directory = os.path.join(*dir_components)
#     new_path = os.path.join(new_directory, filename)
#     audio = new_path
#     label = example["track-instrument-subtype"]
#     return (audio, label)

# with gr.Blocks() as demo:
#     with gr.Column():
#         for _ in range(4):
#             audio, label = generate_audio()
#             print(audio, label)
#             output = gr.Audio(audio, label=label)

# def generate_audio():
#     example = gtzan["train"].shuffle()[0]
#     audio = example["audio"]
#     return (
#         audio["sampling_rate"],
#         audio["array"],
#     ), id2label_fn(example["genre"])


# with gr.Blocks() as demo:
#     with gr.Column():
#         for _ in range(4):
#             audio, label = generate_audio()
#             output = gr.Audio(audio, label=label)

# demo.launch(debug=True)

In [ ]:
!pip install transformers -U

In [ ]:
from transformers import AutoFeatureExtractor

model_id = "ntu-spml/distilhubert"
feature_extractor = AutoFeatureExtractor.from_pretrained(
    model_id, do_normalize=True, return_attention_mask=True
)

In [ ]:
sampling_rate = feature_extractor.sampling_rate
sampling_rate

In [ ]:
!pip install samplerate
!pip install resampy
!pip install librosa

In [ ]:
from datasets import Audio
from librosa import resample
import os
import soundfile as sf
# tracks_train = tracks_train
# gtzan = gtzan.cast_column("audio", Audio(sampling_rate=sampling_rate))


def resample_audio(df, target_samplerate, duration=30):
    # Calculate the number of samples for the desired duration
    num_samples = int(duration * target_samplerate)

    # Create a new column for the resampled audio
    df['resampled_audio'] = None

    for index, row in df.iterrows():
        original_path = '.' + str(row['track-audio-features']['track-audio-path'])
        directory, filename = os.path.split(original_path)
        dir_components = directory.split(os.sep)
        dir_components.insert(2, "Truncated")
        new_directory = os.path.join(*dir_components)
        new_path = os.path.join(new_directory, filename)
        audio = new_path
        print(audio)

        data, samplerate = sf.read(audio, dtype='float32')

        if samplerate != target_samplerate:
            if data.ndim == 2:  # Check if stereo
                left_channel = data[:, 0]
                right_channel = data[:, 1]
                left_resampled = resample(left_channel, orig_sr=samplerate, target_sr=target_samplerate)
                right_resampled = resample(right_channel, orig_sr=samplerate, target_sr=target_samplerate)
                data = np.vstack((left_resampled, right_resampled)).T  # Recombine
            else:  # Mono
                data = resample(data, orig_sr=samplerate, target_sr=target_samplerate)

        # Truncate the audio data to the desired duration
        data = data[:num_samples]

        # Store the resampled and truncated data in the dataframe
        df.at[index, 'resampled_audio'] = {'array': data, 'sampling_rate': target_samplerate}

    return df

duration = 30
# Usage
target_samplerate = 16000  # Define the target sample rate
tracks_train = resample_audio(tracks_train, target_samplerate, duration)
tracks_test = resample_audio(tracks_test, target_samplerate, duration)

tracks_test.to_pickle("./tracks-test-df.pkl")
tracks_train.to_pickle("./tracks-train-df.pkl")

In [ ]:
tracks_train.head(5)

In [ ]:
import numpy as np

sample = tracks_train.iloc[0]["resampled_audio"]

print(f"Mean: {np.mean(sample['array']):.3}, Variance: {np.var(sample['array']):.3}")

In [ ]:
inputs = feature_extractor(sample["array"], sampling_rate=sample["sampling_rate"])

print(f"inputs keys: {list(inputs.keys())}")

print(
    f"Mean: {np.mean(inputs['input_values']):.3}, Variance: {np.var(inputs['input_values']):.3}"
)

In [ ]:
# We have to cut so that everything is the same length (30 seconds)
max_duration = 30.0


# def preprocess_function(examples):
#     audio_arrays = [x["array"] for x in examples["audio"]]
#     inputs = feature_extractor(
#         audio_arrays,
#         sampling_rate=feature_extractor.sampling_rate,
#         max_length=int(feature_extractor.sampling_rate * max_duration),
#         truncation=True,
#         return_attention_mask=True,
#     )
#     return inputs

def preprocess_function(examples):
    audio_arrays = examples["resampled_audio"]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        max_length=int(feature_extractor.sampling_rate * max_duration),
        truncation=True,
        return_attention_mask=True,
    )
    return inputs

In [ ]:
# gtzan_encoded = gtzan.map(
#     preprocess_function,
#     remove_columns=["audio", "file"],
#     batched=True,
#     batch_size=100,
#     num_proc=1,
# )
# gtzan_encoded

# import datasets 

import pandas as pd
import datasets

tracks_test_2 = pd.DataFrame(tracks_test)
# tracks_test_2 = tracks_test_2.drop(columns=['channel-mode', 'parameters', 'track-name', 'track-type', 'track-audio-features'])
# tracks_test_2['track-instrument-subtype'] = tracks_test_2['track-instrument-subtype'].astype('string')
# tracks_test_2['track-instrument-type'] = tracks_test_2['track-instrument-type'].astype('string')
# tracks_test_2['resampled_audio'] = tracks_test_2['resampled_audio'].apply(lambda x: x['array'].flatten())
# test_dataset = datasets.Dataset.from_pandas(tracks_test_2)

# print(tracks_test_dict)

# def data_generator(df, batch_size):
#     for i in range(0, len(df), batch_size):
#         batch_df = df[i:i+batch_size]
#         track_type_batch = list(batch_df['track-instrument-type'])
#         track_subtype_batch = list(batch_df['track-instrument-subtype'])
#         audio_batch = list(batch_df['resampled_audio'])
#         yield {'track-instrument-type': track_type_batch, 'track-instrument-subtype': track_subtype_batch, 'resampled_audio': audio_batch}

# test_dataset = datasets.Dataset.from_generator(
#     data_generator,
#     gen_kwargs={'df': tracks_test_2, 'batch_size': 10}
# )

# Set the desired chunk size
chunk_size = 10

# Initialize an empty list to store the processed chunks
processed_chunks = []

# Iterate over the DataFrame in chunks
for i in range(0, len(tracks_test_2), chunk_size):
    chunk = tracks_test_2.iloc[i:i+chunk_size]
    
    # Drop the specified columns
    chunk = chunk.drop(columns=['channel-mode', 'parameters', 'track-name', 'track-type', 'track-audio-features'])
    
    # Convert 'track-instrument-subtype' and 'track-instrument-type' to string type
    chunk['track-instrument-subtype'] = chunk['track-instrument-subtype'].astype('string')
    chunk['track-instrument-type'] = chunk['track-instrument-type'].astype('string')
    
    # Process the 'resampled_audio' column
    chunk['resampled_audio'] = chunk['resampled_audio'].apply(lambda x: x['array'].ravel())
    
    # Append the processed chunk to the list
    processed_chunks.append(chunk)

# Concatenate the processed chunks into a single DataFrame
tracks_test_2 = pd.concat(processed_chunks, ignore_index=True)

# Create a dictionary to store the data
data_dict = {
    'track-instrument-subtype': list(tracks_test_2['track-instrument-subtype']),
    'track-instrument-type': list(tracks_test_2['track-instrument-type']),
    'resampled_audio': list(tracks_test_2['resampled_audio'])
}

# Create the dataset using from_dict()
test_dataset = datasets.Dataset.from_dict(data_dict)

# combined_df = pd.concat([tracks_train, tracks_test], ignore_index=True)
# combined_df['split'] = ['train'] * len(tracks_train) + ['test'] * len(tracks_test)
# tracks_train = tracks_train.drop(columns=['channel-mode', 'parameters', 'track-name', 'track-type', 'track-audio-features'])
# tracks_train['resampled_audio'] = tracks_train['resampled_audio'].astype('float32')
# tracks_train['track-instrument-subtype'] = tracks_train['track-instrument-subtype'].astype('string')
# tracks_train['track-instrument-type'] = tracks_train['track-instrument-type'].astype('string')
# tracks_train['resampled_audio'] = tracks_train['resampled_audio'].apply(lambda x: x['array'])
# tracks_train['resampled_audio'] = np.array(tracks_train['resampled_audio']).flatten()
# tracks_train['resampled_audio'] = np.array(tracks_train['resampled_audio'])
# print(tracks_train.iloc[0]['resampled_audio'])
# print(tracks_train.dtypes)
# tracks_train.head(5)
# hf_dataset = datasets.Dataset.from_pandas(tracks_train)

# processed_dataset = hf_dataset.map(
#     preprocess_function,
#     remove_columns=["columns_to_remove"],  # Replace with actual column names
#     batched=True,
#     batch_size=100,
#     num_proc=1  # Adjust num_proc if you want to use parallel processing
# )



In [ ]:
test_dataset[0]

In [ ]:
tracks_train_2 = pd.DataFrame(tracks_train)
# tracks_train_2 = tracks_train_2.drop(columns=['channel-mode', 'parameters', 'track-name', 'track-type', 'track-audio-features'])
# tracks_train_2['track-instrument-subtype'] = tracks_train_2['track-instrument-subtype'].astype('string')
# tracks_train_2['track-instrument-type'] = tracks_train_2['track-instrument-type'].astype('string')
# tracks_train_2['resampled_audio'] = tracks_train_2['resampled_audio'].apply(lambda x: x['array'].ravel())
# # tracks_train_dict = tracks_train_2.to_dict('list')
# train_dataset = datasets.Dataset.from_pandas(tracks_train_2)

# Set the desired chunk size
chunk_size = 10

# Initialize an empty list to store the processed chunks
processed_chunks = []

# Iterate over the DataFrame in chunks
for i in range(0, len(tracks_train_2), chunk_size):
    chunk = tracks_train_2.iloc[i:i+chunk_size]
    
    # Drop the specified columns
    chunk = chunk.drop(columns=['channel-mode', 'parameters', 'track-name', 'track-type', 'track-audio-features'])
    
    # Convert 'track-instrument-subtype' and 'track-instrument-type' to string type
    chunk['track-instrument-subtype'] = chunk['track-instrument-subtype'].astype('string')
    chunk['track-instrument-type'] = chunk['track-instrument-type'].astype('string')
    
    # Process the 'resampled_audio' column
    chunk['resampled_audio'] = chunk['resampled_audio'].apply(lambda x: x['array'].ravel())
    
    # Append the processed chunk to the list
    processed_chunks.append(chunk)

# Concatenate the processed chunks into a single DataFrame
tracks_train_2 = pd.concat(processed_chunks, ignore_index=True)

# Create a dictionary to store the data
train_dict = {
    'track-instrument-subtype': list(tracks_train_2['track-instrument-subtype']),
    'track-instrument-type': list(tracks_train_2['track-instrument-type']),
    'resampled_audio': list(tracks_train_2['resampled_audio'])
}

# Create the dataset using from_dict()
train_dataset = datasets.Dataset.from_dict(train_dict)

In [ ]:
dataset_cc = datasets.concatenate_datasets([train_dataset, test_dataset])

In [ ]:
dataset_cc.shuffle(100)
dataset_cc = dataset_cc.train_test_split(0.2)

In [ ]:
# Load dataset
from datasets import load_dataset
music_mixing = load_dataset("mclemcrew/mix-tracks", "default")

In [ ]:
# gtzan_encoded = gtzan_encoded.rename_column("genre", "label")
# dataset_cc = dataset_cc.rename_column("track-instrument-type", "label")
music_mixing['test'][0]
# dataset_cc = music_mixing.train_test_split(seed=42, shuffle=True, test_size=0.1)

In [ ]:
music_mixing

In [ ]:
# Extract unique labels
test_dataset = music_mixing
unique_labels = test_dataset["train"].unique("label")

# Create label mappings
label2id = {label: i for i, label in enumerate(unique_labels)}
id2label = {i: label for label, i in label2id.items()}

def prepare_dataset(examples):
    audio = examples["resampled_audio"]
    normalized_examples = feature_extractor(audio, sampling_rate=feature_extractor.sampling_rate)
    normalized_examples['label'] = [label2id[label] for label in examples['label']]
    return normalized_examples

In [ ]:
# Prepare for training
dataset = test_dataset.map(prepare_dataset, batched=True)  

In [ ]:
from datasets import Dataset

# function to get unique values
def unique(list_object):
 
    # initialize a null list
    unique_list = []
 
    # traverse for all elements
    for x in list_object:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)
    return unique_list
        
# Get all unique labels
unique_labels = unique(dataset_cc["train"]["label"])

print(unique_labels)
print(dataset_cc["train"][0])

# Create the mapping
# label_to_int = {label: i for i, label in enumerate(unique_labels)}

# def convert_label_to_int(example):
#     example["label"] = label_to_int[example["label"]]
#     return example

# dataset = dataset_cc.map(convert_label_to_int, batched=True)

In [ ]:
# # id2label = {
# #     str(i): id2label_fn(i)
# #     for i in range(len(dataset_cc["train"].features["label"].keys()))
# # }
# # label2id = {v: k for k, v in id2label.items()}

# # id2label["7"]

# from datasets import ClassLabel

# # Assuming dataset_cc["train"].features["label"] is a ClassLabel object
# # Find the unique labels in your dataset
# unique_labels = set(dataset_cc["train"]["label"])

# # Create the id2label dictionary
# id2label = {
#     str(i): id2label_fn(label)
#     for i, label in enumerate(unique_labels)
# }

# # Create the label2id dictionary
# label2id = {v: k for k, v in id2label.items()}

## Training Portion

In [ ]:
from transformers import AutoModelForAudioClassification

num_labels = len(id2label)

model = AutoModelForAudioClassification.from_pretrained(
    model_id,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
)

In [ ]:
from transformers import TrainingArguments

model_name = model_id.split("/")[-1]
batch_size = 8
gradient_accumulation_steps = 1
num_train_epochs = 10

training_args = TrainingArguments(
    f"{model_name}-finetuned-gtzan",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    warmup_ratio=0.1,
    logging_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
)

In [ ]:
!pip install evaluate

In [ ]:
import evaluate
import numpy as np

metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [ ]:
!pip install ipywidgets

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=gtzan_encoded["train"],
    eval_dataset=gtzan_encoded["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()